In [1]:
import os
import pandas as pd
import re
from sqlalchemy import create_engine

In [2]:
xls_sku_dtype = {"skuId":"int64"
    ,"sku名称":"object"
    ,"支付金额":"object"
    ,"支付买家数":"object"
    ,"支付件数":"object"
    ,"加购件数":"object"
                            }

In [3]:
def df_to_sql(df_to_dwh,dwh_list):
    username = "root"
    password = "9518llhh731X"
    host = "localhost"
    port = 3306
    database = "dwh"
    charset = "utf8mb4"
    connection_string = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}?charset={charset}'
    conn = create_engine(connection_string)
    pd.io.sql.to_sql(df_to_dwh,dwh_list,con=conn,if_exists="append",index=False)
    return

In [4]:
skuList = ["塞德思","希纳露","百热朗","健眠计划","棒维素","琵力消","整肠药"]
def sku_tag_add(itemName,itemLists):
    for item in itemLists:
        if item in itemName:
            return item
        else:
            continue
    return '灵的融'

In [5]:
def get_sku_sales_date(xls_file_name):
    date_pattern = r"\d{4}-\d{2}-\d{2}"
    sku_date = re.findall(date_pattern,xls_file_name)[0]
    return sku_date

In [6]:
sku_sales_xls_list=os.listdir(r"D:\盐野义\数据\sku_data")

In [7]:
sku_data = pd.read_excel(io="D:/盐野义/数据/sku_data/"+sku_sales_xls_list[0],skiprows=5,engine="xlrd",dtype=xls_sku_dtype)
sku_data["日期"] = pd.to_datetime(get_sku_sales_date(sku_sales_xls_list[0]))

In [8]:
for xls_file in sku_sales_xls_list[1:]:
    sku_data_sub = pd.read_excel(io="D:/盐野义/数据/sku_data/"+xls_file,skiprows=5,engine="xlrd",dtype=xls_sku_dtype)
    sku_data_sub["日期"] = pd.to_datetime(get_sku_sales_date(xls_file))
    sku_data = pd.concat([sku_data,sku_data_sub],axis=0,ignore_index=True)

In [9]:
sku_data["商品别名"] = sku_data["sku名称"].apply(lambda x:sku_tag_add(x,itemLists=skuList))


In [10]:
sku_data["支付金额"] = sku_data["支付金额"].apply(lambda x:x.replace(",",""))
sku_data["支付买家数"] = sku_data["支付买家数"].apply(lambda x:x.replace(",",""))
sku_data["支付件数"] = sku_data["支付件数"].apply(lambda x:x.replace(",",""))
sku_data["加购件数"] = sku_data["加购件数"].apply(lambda x:x.replace(",",""))

In [11]:
sku_data = sku_data.astype({"日期":"datetime64[ns]"
    ,"skuId":"int64"
    ,"sku名称":"object"
    ,"支付金额":"float64"
    ,"支付买家数":"int64"
    ,"支付件数":"int64"
    ,"加购件数":"int64"
                            })


In [12]:
df_to_sql(df_to_dwh=sku_data,dwh_list="yyy_sku")

In [15]:
username = "root"
password = "9518llhh731X"
host = "localhost"
port = 3306
database = "dwh"
charset = "utf8mb4"
connection_string = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}?charset={charset}'
conn = create_engine(connection_string)
query = '''
select * from 
((
select 
sales.统计日期 as 日期
,sales.商品别名 
,'' as skuid
,'' as sku名称
,sum(sales.商品访客数) as 商品访客数
,sum(sales.商品浏览量) as 商品浏览量
,avg(sales.平均停留时长) as 商品平均停留时长
,avg(sales.商品详情页跳出率) as 商品详情页跳出率
,sum(sales.商品加购人数) as 商品加购人数
,sum(sales.商品加购件数) as 商品加购件数
,sum(sales.商品收藏人数) as 商品收藏人数
,sum(sales.下单买家数) as 下单买家数
,sum(sales.下单金额) as 下单金额
,avg(sales.下单转化率) as 下单转化率
,sum(sales.支付买家数) as 支付买家数
,sum(sales.下单件数) as 下单件数
,avg(sales.商品支付转化率) as 支付转化率
,sum(sales.支付金额) as 支付金额
,sum(sales.支付件数) as 支付件数
,sum(sales.聚划算支付金额) as 聚划算支付金额
,sum(sales.支付新买家数) as 支付新买家数
,sum(sales.支付老买家数) as 支付老买家数
,sum(sales.老买家支付金额) as 老买家支付金额
,max(sales.月累计支付金额) as 月累计支付金额
,max(sales.月累计支付件数) as 月累计支付件数
,max(sales.年累计支付金额) as 年累计支付金额
,avg(sales.访客平均价值) as 访客平均价值
,sum(sales.成功退款金额) as 成功退款金额
from yyy_sales sales
where sales.统计日期 >='2024-08-26'
group by sales.统计日期  ,sales.商品别名,skuid,sku名称)

union all

(select 
sku.日期 
,sku.商品别名 as 商品别名
,max(sku.skuId) as skuId 
,sku.sku名称 
,null as 商品访客数
,null as 商品浏览量
,null as 商品平均停留时长
,null as 商品详情页跳出率
,null as 商品加购人数
,sum(sku.加购件数) as 商品加购件数
,null as 商品收藏人数
,null as 下单买家数
,null as 下单金额
,null as 下单转化率
,sum(sku.支付买家数) as 支付买家数
,null as 下单件数
,null as 支付转化率
,sum(sku.支付金额) as 支付金额
,sum(sku.支付件数) as 支付件数
,null as 聚划算支付金额
,null as 支付新买家数
,null as 支付老买家数
,null as 老买家支付金额
,null as 月累计支付金额
,null as 月累计支付件数
,null as 年累计支付金额
,null as 访客平均价值
,null as 成功退款金额
from yyy_sku sku
where sku.日期 >='2024-08-26'
group by sku.日期
,sku.商品别名
,sku.sku名称 
,商品访客数
,商品浏览量
,商品平均停留时长
,商品详情页跳出率
,商品加购人数
,商品收藏人数
,下单买家数
,下单金额
,下单转化率
,下单件数
,支付转化率
,聚划算支付金额
,支付新买家数
,支付老买家数
,老买家支付金额
,月累计支付金额
,月累计支付件数
,年累计支付金额
,访客平均价值
,成功退款金额)) as temp
where 商品别名 in ("灵的融","希纳露","百热朗","塞德思")
order by 商品别名,日期

'''
df = pd.read_sql_query(query, conn)
 
conn.dispose(close=True)

In [16]:
with pd.ExcelWriter("D:/盐野义/数据/sku_data/sku_all.xlsx") as wt:    
    for _item in ["灵的融","希纳露","百热朗","塞德思"]:
        df_sub = df[df["商品别名"]==_item]
        df_sub.to_excel(wt,index=False,sheet_name=_item)